<a href="https://colab.research.google.com/github/beza-lab/KAIM-WEEK-5/blob/main/task3_fine_tune_NER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers datasets pandas

In [6]:
import pandas as pd

# Reading the .conll file into a DataFrame
dataset_path = '/content/dataset (1).conll'
# Make sure this file path is correct and the file exists in this path
data = pd.read_csv(dataset_path, sep='\t', header=None, skip_blank_lines=True)

# Check the first few rows of the dataset
print(data.head())


              0
0  ⬇️⬇️⬇️9pcs O
1     Kitchen O
2       knife O
3         Set O
4         ✔️1 O


In [4]:
import re
import pandas as pd
from itertools import groupby

# Load the dataset
dataset_path = '/content/dataset.conll'
data = pd.read_csv(dataset_path, sep='\t', header=None, skip_blank_lines=False, names=['token', 'label'])

# Define a function to clean and parse the dataset
def clean_and_parse_data(data):
    tokens, labels = [], []
    token_seq, label_seq = [], []

    for row in data.itertuples(index=False):
        if pd.isna(row.token):
            if token_seq:  # Append token/label sequence only if it exists
                tokens.append(token_seq)
                labels.append(label_seq)
                token_seq, label_seq = [], []
        else:
            token = re.sub(r'[^A-Za-z0-9ሀ-መ-\s]', '', str(row.token))
            label = row.label if row.label else 'O'
            token_seq.append(token)
            label_seq.append(label)

    # Ensure the last sequence is appended
    if token_seq:
        tokens.append(token_seq)
        labels.append(label_seq)

    return {'tokens': tokens, 'ner_tags': labels}

# Clean and parse dataset
dataset_dict = clean_and_parse_data(data)

# Converting dictionary to Hugging Face Dataset
from datasets import Dataset

dataset = Dataset.from_dict(dataset_dict)

print(dataset)


Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 14
})


In [5]:
from transformers import AutoTokenizer

# Initialize the tokenizer for the selected model
model_name = 'xlm-roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        padding='max_length',
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return {k: v for k, v in tokenized_inputs.items()}

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
from datasets import DatasetDict

# Create a DatasetDict with train and validation splits
split_dataset = DatasetDict({
    'train': dataset.select(range(12)),  # Assume 12 rows for training
    'val': dataset.select(range(12, 14))  # Assume last 2 rows for validation
})

print(split_dataset)


DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 12
    })
    val: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2
    })
})


In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['tokens'],
        truncation=True,
        padding='max_length',
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return {k: v for k, v in tokenized_inputs.items()}

tokenized_dataset = split_dataset.map(tokenize_and_align_labels, batched=True)


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [9]:
from transformers import Trainer, AutoModelForTokenClassification

num_labels = len(set(sum([tags for tags in dataset['ner_tags']], [])))

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['val'],
    tokenizer=tokenizer,
)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-871d9f762d0c>:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: gahebs2 (gahebs2-10-academy). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
